In [16]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent
from langchain_community.chat_models import ChatOpenAI

import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)

question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    max_steps=7,
    max_trials=3,
)

In [17]:
from typing import List 

def gather_experience(
    reflexion_react_agent: ReflexionReActAgent,
    questions: List[str],
    keys: List[str],
    strategy: str = "reflexion",
):
    out = {
        "idx": [],
        "questions": [],
        "keys": [],
        "trajectories": [],
        "reflections": []
    }
    for idx, (question, key) in enumerate(zip(questions, keys)):
        out["idx"].append(idx)
        out["questions"].append(question)
        out["keys"].append(key)
        trajs = reflexion_react_agent.generate(
            question=question, key=key, strategy=strategy
        )
        out["trajectories"].append(trajs)
    return out

In [18]:
out = gather_experience(agent, questions=[question], keys=[key])
out

{'idx': [0],
 'questions': ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"],
 'keys': ['Gesellschaft mit beschränkter Haftung'],
 'trajectories': [["\nThought: I need to search for VIVA Media AG to find out their new acronym and what it stands for.\nAction: Search[VIVA Media AG]\nObservation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'VIVA Plus', 'Paramount International Networks', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'GfK Entertainment charts', 'Lindt', 'Spellbound Entertainment', 'List of multinational corporations']\nThought: I should try searching for VIVA Media AG on Wikipedia to get more information, as the initial search did not yield relevant results.\nAction: Search[VIVA Media AG Wikipedia]\nObservation 2: Could not find [VIVA Media AG Wikipedia]. Similar: ['VIVA (German TV channel)', 'Spellbound Entertainment', 'Julie Meyer', 'Opel Astra', 'Sildenafil', 'Brainpool TV', 'Mobile 

: 

In [13]:
agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    max_steps=7,
    max_trials=3,
)
trajs = agent.generate(
    question=question, key=key, strategy="reflexion"
)

In [14]:
trajs

['\nThought: I need to search for VIVA Media AG and find out what their new acronym stands for after changing their name in 2004. Let\'s start by searching for VIVA Media AG.\nAction: Search[VIVA Media AG]\nObservation 1: Could not find [VIVA Media AG]. Similar: [\'MTV Music (Polish TV channel)\', \'VIVA Plus\', \'Paramount International Networks\', \'VIVA (German TV channel)\', \'Viacom (1952–2005)\', \'Vauxhall Viva\', \'GfK Entertainment charts\', \'Lindt\', \'Spellbound Entertainment\', \'List of multinational corporations\']\nThought: I couldn\'t find VIVA Media AG, but there are similar entities like VIVA (German TV channel) and VIVA Plus. Let\'s try searching for VIVA (German TV channel) to see if we can find information about the name change and their new acronym.\nAction: Search[VIVA (German TV channel)]\nObservation 2: VIVA (abbreviation for: Videoverwertungsanstalt translating to "Video Exploitation Facility") was a German free-to-air music television channel, first broadcas